In [ ]:
#| default_exp handlers.maris_dump

# MARIS dump
> Data pipeline (handler) to convert global MARIS db dump into `NetCDF` format

The input data is a dump from already imported MARIS datasets.


**Questions**:
1. is `decayedto` used and in what context? (question to be answered for all columns actually)
2. what are units of uncertainty?
3. ref_id vs. zoterourl vs. displaytext
   1.  e.g ref_id=129 and 130 points to the same dataset?

**Dev. board**: https://trello.com/b/IszgV1bj/marisco

## Packages import

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export
from tqdm import tqdm
from pathlib import Path
import fastcore.all as fc
import pandas as pd

from marisco.callbacks import (Callback, Transformer, SanitizeLonLatCB)
from marisco.metadata import (GlobAttrsFeeder, BboxCB,
                              DepthRangeCB, TimeRangeCB,
                              ZoteroCB, KeyValuePairCB)
from marisco.configs import lut_path, cdl_cfg, cfg, nc_tpl_path
from marisco.serializers import NetCDFEncoder

In [ ]:
pd.set_option('display.max_rows', 100)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
fname_in = Path().home() / 'pro/data/maris/all-maris.txt'
dir_dest = '../../_data/output/dump'
# fname_out = '../../_data/output/helcom.nc'

## Utils

In [ ]:
# | export
def load_dump(fname): 
    return pd.read_csv(fname, sep='\t', encoding='ISO-8859-1')   

def load_data(df:pd.DataFrame, # MARIS global dump 
                 ref_id:int, # Reference id of interest
                 ):
    "Load specific MARIS dataset through its ref_id."
    lut = {
        'Sediment': 'sediment',
        'Seawater': 'seawater',
        'Suspended matter': 'suspended-matter',
        'Biota': 'biota'}
    dfs = {}
    for name, grp in df[df.ref_id  == ref_id].groupby('samptype'):
        dfs[lut[name]] = grp
    return dfs

def get_zotero_key(dfs):
    return dfs[next(iter(dfs))][['zoterourl']].iloc[0].values[0].split('/')[-1]

def get_fname(dfs):
    id, name = dfs[next(iter(dfs))][['ref_id', 'displaytext']].iloc[0]
    name = name.replace(',', '').replace('.', '').replace('-', ' ').split(' ')
    return '-'.join(([str(id)] + name)) + '.nc'

## Load data

In [ ]:
df = load_dump(fname_in)

In [ ]:
# 52, 191 (OSPAR)
ref_id = 191

## Data transformation pipeline

### Normalize nuclide names

In [ ]:
#| export
def get_varnames_lut():
    fname = lut_path() / 'dbo_nuclide.xlsx'
    df_nuclide = pd.read_excel(fname, usecols=['nuclide_id', 'nc_name'])
    return df_nuclide.set_index('nuclide_id').to_dict()['nc_name']

In [ ]:
# | export
class RemapRdnNameCB(Callback):
    "Remap to MARIS radionuclide names."
    def __init__(self,
                 fn_lut=get_varnames_lut):
        fc.store_attr()

    def __call__(self, tfm):
        lut = self.fn_lut()
        for k in tfm.dfs.keys():
            tfm.dfs[k]['nuclide_id'] = tfm.dfs[k]['nuclide_id'].replace(lut)

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[RemapRdnNameCB()])

print(tfm()['biota']['nuclide_id'].unique())

['cs137' 'tc99' 'pu239_240_tot' 'po210' 'ra226' 'ra228' 'pb210' 'pu238'
 'am241' 'c14' 'k40' 'i131']


### Rename columns

In [ ]:
dfs['biota'].columns

Index(['sample_id', 'area_id', 'areaname', 'samptype_id', 'samptype', 'ref_id',
       'displaytext', 'zoterourl', 'ref_note', 'datbase', 'lab_id', 'lab',
       'latitude', 'longitude', 'begperiod', 'endperiod', 'samplingyear',
       'totdepth', 'sampdepth', 'station', 'samplabcode', 'species_id',
       'taxonname', 'taxonrank', 'biogroup', 'biogroup_id', 'taxondb',
       'taxondbid', 'taxondburl', 'taxonrepname', 'bodypar_id', 'bodypar',
       'sliceup', 'slicedown', 'sedtype_id', 'sedtype', 'sedrepname',
       'nuclide_id', 'nusymbol', 'volume', 'salinity', 'temperatur',
       'filtered', 'filtpore', 'samparea', 'drywt', 'wetwt', 'percentwt',
       'sampmet_id', 'sampmet', 'prepmet_id', 'prepmet', 'drymet_id', 'drymet',
       'counmet_id', 'counmet', 'decayedto', 'detection', 'activity',
       'uncertaint', 'unit_id', 'unit', 'vartype', 'freq', 'rangelow',
       'rangeupp', 'profile', 'transect_id', 'measure_note', 'shapetype_id',
       'profile_id', 'sampnote', 'ref_full

In [ ]:
#| export
# To be added: endperiod, totdepth, lab
def renaming_rules():
    vars = cdl_cfg()['vars']
    # Define column names renaming rules
    return {
        'latitude': vars['defaults']['lat']['name'],
        'longitude': vars['defaults']['lon']['name'],
        'begperiod': vars['defaults']['time']['name'],
        'sampdepth': vars['defaults']['depth']['name'],
        'uncertaint': vars['suffixes']['uncertainty']['name'],
        'unit_id': vars['suffixes']['unit']['name'],
        'detection': vars['suffixes']['detection_limit']['name'],
        'area_id': vars['defaults']['area']['name'], 
        'species_id': vars['bio']['species']['name'],
        'biogroup_id': vars['bio']['bio_group']['name'],
        'bodypar_id': vars['bio']['body_part']['name'],
        'sedtype_id': vars['sed']['sed_type']['name'],
        'volume': vars['suffixes']['volume']['name'],
        'salinity': vars['suffixes']['salinity']['name'],
        'temperatur': vars['suffixes']['temperature']['name'],
        'sampmet_id': vars['suffixes']['sampling_method']['name'],
        'prepmet_id': vars['suffixes']['preparation_method']['name'],
        'counmet_id': vars['suffixes']['counting_method']['name'],
        'activity': 'value',
        'nuclide_id': 'nuclide'
    }

In [ ]:
#| export
class RenameColumnCB(Callback):
    "Renaming variables to MARIS standard names."
    def __init__(self,
                 renaming_rules=renaming_rules):
        fc.store_attr()

    def __call__(self, tfm):
        lut = renaming_rules()
        coi = lut.keys()
        for k in tfm.dfs.keys():
            # Select cols of interest
            tfm.dfs[k] = tfm.dfs[k].loc[:, coi]
            # Rename cols
            tfm.dfs[k].rename(columns=lut, inplace=True)

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB()
    ])

print(tfm()['biota'])

              lat        lon     time  depth       unc  unit dl  area  \
444800  64.233333 -14.833333  00:00.0    NaN       NaN     5  <  1912   
444801  64.133333 -15.800000  00:00.0    NaN  0.023629     5  =  1912   
444802  65.916667 -24.850000  00:00.0    NaN  0.020879     5  =  1912   
444803  63.350000 -19.166667  00:00.0    NaN  0.017631     5  =  1912   
444804  63.800000 -21.133333  00:00.0    NaN  0.016551     5  =  1912   
...           ...        ...      ...    ...       ...   ... ..   ...   
816812  70.274722  30.945556  00:00.0    NaN       NaN     5  <  9999   
816813  70.274722  30.945556  00:00.0    NaN       NaN     5  <  9999   
816814  70.486389 -21.976944  00:00.0    NaN  0.013810     5  =  9999   
816815  74.469722  18.969722  00:00.0    NaN  0.020000     5  =  9999   
816816  74.469722  18.969722  00:00.0    NaN  0.050000     5  =  9999   

        species  bio_group  body_part  sed_type  vol  sal  temp  sampmet  \
444800      270          4         52         0

### Drop NaN only columns

In [ ]:
#| export
class DropNAColumnsCB(Callback):
    "Drop variable containing only NaN or 'Not available' (id=0 in MARIS lookup tables)."
    def __init__(self, na_value=0):
        fc.store_attr()
        
    def isMarisNA(self, col): 
        return len(col.unique()) == 1 and col.iloc[0] == self.na_value
    
    def dropMarisNA(self, df):
        na_cols = [col for col in df.columns if self.isMarisNA(df[col])]
        return df.drop(labels=na_cols, axis=1)
        
    def __call__(self, tfm):
        for k in tfm.dfs.keys():
            tfm.dfs[k] = tfm.dfs[k].dropna(axis=1, how='all')
            tfm.dfs[k] = self.dropMarisNA(tfm.dfs[k])

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB(),
    DropNAColumnsCB()
    ])

print(tfm()['biota'])

              lat        lon     time       unc  unit dl  area  species  \
444800  64.233333 -14.833333  00:00.0       NaN     5  <  1912      270   
444801  64.133333 -15.800000  00:00.0  0.023629     5  =  1912      270   
444802  65.916667 -24.850000  00:00.0  0.020879     5  =  1912      270   
444803  63.350000 -19.166667  00:00.0  0.017631     5  =  1912      270   
444804  63.800000 -21.133333  00:00.0  0.016551     5  =  1912      270   
...           ...        ...      ...       ...   ... ..   ...      ...   
816812  70.274722  30.945556  00:00.0       NaN     5  <  9999       96   
816813  70.274722  30.945556  00:00.0       NaN     5  <  9999       96   
816814  70.486389 -21.976944  00:00.0  0.013810     5  =  9999        0   
816815  74.469722  18.969722  00:00.0  0.020000     5  =  9999       99   
816816  74.469722  18.969722  00:00.0  0.050000     5  =  9999       99   

        bio_group  body_part     value nuclide  
444800          4         52  0.063508   cs137  
4

### Sanitize detection limit values

In [ ]:
#| export
def get_dl_lut():
    fname = lut_path() / 'dbo_detectlimit.xlsx'
    df_nuclide = pd.read_excel(fname, usecols=['name', 'id'])
    return df_nuclide.set_index('name').to_dict()['id']

In [ ]:
get_dl_lut()

{'Not Available': 0, '=': 1, '<': 2, 'ND': 3, 'DE': 4}

In [ ]:
#| export
class SanitizeDetectionLimitCB(Callback):
    "Assign Detection Limit name to its id based on MARIS nomenclature."
    def __init__(self,
                 fn_lut=get_dl_lut):
        fc.store_attr()

    def __call__(self, tfm):
        lut = self.fn_lut()
        for k in tfm.dfs.keys():
            tfm.dfs[k]['dl'] = tfm.dfs[k]['dl'].replace(lut)

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB(),
    DropNAColumnsCB(),
    SanitizeDetectionLimitCB()
    ])

# print(tfm()['sediment'])
df_test = tfm()['biota']

In [ ]:
df_test.columns

Index(['lat', 'lon', 'time', 'unc', 'unit', 'dl', 'area', 'species',
       'bio_group', 'body_part', 'value', 'nuclide'],
      dtype='object')

In [ ]:
df_test.dl.unique()

array([2, 1, 3])

### Reshape: long to wide

In [ ]:
#| export
class ReshapeLongToWide(Callback):
    "Convert data from long to wide with renamed columns."
    def __init__(self, value_col='nuclide'):
        fc.store_attr()
        self.derived_cols = [value['name'] for value in cdl_cfg()['vars']['suffixes'].values()]
    
    def renamed_cols(self, cols):
        return [f'{inner}_{outer}' if inner else outer for outer, inner in cols]

    def pivot(self, df):
        derived_coi = [col for col in self.derived_cols if col in df.columns]
        df.reset_index(names='sample', inplace=True)
        idx = list(set(df.columns) - set([self.value_col] + derived_coi))
        return df.pivot_table(index=idx,
                              columns=self.value_col,
                              values=derived_coi,
                              fill_value=0,
                              aggfunc=lambda x: x
                              ).reset_index()

    def __call__(self, tfm):
        for k in tfm.dfs.keys():
            tfm.dfs[k] = self.pivot(tfm.dfs[k])
            tfm.dfs[k].columns = self.renamed_cols(tfm.dfs[k].columns)
            # tfm.dfs[k].index.name = 'sample'
            tfm.dfs[k].set_index('sample', inplace=True)

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB(),
    DropNAColumnsCB(),
    SanitizeDetectionLimitCB(),
    ReshapeLongToWide()
    ])

print(tfm()['biota'])

        bio_group  area  species  body_part   value     time      lat  \
sample                                                                  
483819          0  2350        0          1   0.165  00:00.0  51.7375   
483820          0  2350        0          1   0.130  00:00.0  51.7375   
483821          0  2350        0          1   2.800  00:00.0  51.7375   
483822          0  2350        0          1  19.000  00:00.0  51.7375   
483823          0  2350        0          1   4.200  00:00.0  51.7375   
...           ...   ...      ...        ...     ...      ...      ...   
815945         14  9999      129         19  19.600  00:00.0  54.0500   
815951         14  9999      129         19   9.750  00:00.0  54.0500   
815957         14  9999      129         19  19.500  00:00.0  54.0500   
815963         14  9999      129         19   6.210  00:00.0  54.0500   
815971         14  9999      129         19  13.400  00:00.0  54.0500   

             lon  am241_dl  c14_dl  ...  cs137_uni

### Encode time (seconds since ...)

In [ ]:
#| export
class EncodeTimeCB(Callback):
    "Encode time as `int` representing seconds since xxx (TBD)."  
    def __call__(self, tfm):
        for k in tfm.dfs.keys():
            tfm.dfs[k].time = 9999

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB(),
    DropNAColumnsCB(),
    SanitizeDetectionLimitCB(),
    ReshapeLongToWide(),
    EncodeTimeCB()
    ])

print(tfm()['biota'])

        bio_group  area  species  body_part   value  time      lat       lon  \
sample                                                                         
483819          0  2350        0          1   0.165  9999  51.7375  2.658333   
483820          0  2350        0          1   0.130  9999  51.7375  2.658333   
483821          0  2350        0          1   2.800  9999  51.7375  2.658333   
483822          0  2350        0          1  19.000  9999  51.7375  2.658333   
483823          0  2350        0          1   4.200  9999  51.7375  2.658333   
...           ...   ...      ...        ...     ...   ...      ...       ...   
815945         14  9999      129         19  19.600  9999  54.0500 -6.200000   
815951         14  9999      129         19   9.750  9999  54.0500 -6.200000   
815957         14  9999      129         19  19.500  9999  54.0500 -6.200000   
815963         14  9999      129         19   6.210  9999  54.0500 -6.200000   
815971         14  9999      129        

### Sanitize coordinates

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB(),
    DropNAColumnsCB(),
    SanitizeDetectionLimitCB(),
    ReshapeLongToWide(),
    EncodeTimeCB(),
    SanitizeLonLatCB()
    ])

# print(tfm()['sediment'])
df_debug = tfm()['biota']

In [ ]:
# pu239_240_tot_dl

In [ ]:
df_debug.species.unique()

array([   0, 1608,   23,  270,   99,  436,  384,  389,  139,  192,  244,
        378,   50,  381,  379,  437,  412,  415,  419,  405,  409,  426,
        428, 1609,  421,  429,  404,  385,  386,  397,  398,  399,  243,
        294,  191, 1606,  443,  444,  393,  380,  416,  432,  433,  439,
        440,  441,  382,  383,  387,  388,  402,  420,  422,  423,  427,
        390,  407,  442,  272,  410,  411,  438, 1441, 1610, 1605,  424,
        395,  401,   96,  435,  392,  400,  430,  417,  396,  434,  734,
        408,  394,  391,  425,  413, 1607,  418,  129,  414,  377,  403,
        274,  431,  406])

## Encode to NetCDF

In [ ]:
dfs = load_data(df, ref_id)
tfm = Transformer(dfs, cbs=[
    RemapRdnNameCB(),
    RenameColumnCB(),
    DropNAColumnsCB(),
    SanitizeDetectionLimitCB(),
    ReshapeLongToWide(),
    EncodeTimeCB(),
    SanitizeLonLatCB()
    ])

dfs_tfm = tfm()

In [ ]:
tfm.logs

['Remap to MARIS radionuclide names.',
 'Renaming variables to MARIS standard names.',
 "Drop variable containing only NaN or 'Not available' (id=0 in MARIS lookup tables).",
 'Assign Detection Limit name to its id based on MARIS nomenclature.',
 'Convert data from long to wide with renamed columns.',
 'Encode time as `int` representing seconds since xxx (TBD).',
 'Drop row when both longitude & latitude equal 0.']

In [ ]:
#| export
kw = ['oceanography', 'Earth Science > Oceans > Ocean Chemistry> Radionuclides',
      'Earth Science > Human Dimensions > Environmental Impacts > Nuclear Radiation Exposure',
      'Earth Science > Oceans > Ocean Chemistry > Ocean Tracers, Earth Science > Oceans > Marine Sediments',
      'Earth Science > Oceans > Ocean Chemistry, Earth Science > Oceans > Sea Ice > Isotopes',
      'Earth Science > Oceans > Water Quality > Ocean Contaminants',
      'Earth Science > Biological Classification > Animals/Vertebrates > Fish',
      'Earth Science > Biosphere > Ecosystems > Marine Ecosystems',
      'Earth Science > Biological Classification > Animals/Invertebrates > Mollusks',
      'Earth Science > Biological Classification > Animals/Invertebrates > Arthropods > Crustaceans',
      'Earth Science > Biological Classification > Plants > Macroalgae (Seaweeds)']

In [ ]:
#| export
def get_attrs(tfm, zotero_key='26VMZZ2Q', kw=kw):
    return GlobAttrsFeeder(tfm.dfs, cbs=[
        BboxCB(),
        DepthRangeCB(),
        TimeRangeCB(cfg()),
        ZoteroCB(zotero_key, cfg=cfg()),
        KeyValuePairCB('keywords', ', '.join(kw)),
        KeyValuePairCB('publisher_postprocess_logs', ', '.join(tfm.logs))
        ])()

In [ ]:
get_attrs(tfm, zotero_key='3W354SQG', kw=kw)

{'geospatial_lat_min': '49.43222222',
 'geospatial_lat_max': '81.26805556',
 'geospatial_lon_min': '-56.15',
 'geospatial_lon_max': '36.18166667',
 'geospatial_bounds': 'POLYGON ((-56.15 36.18166667, 49.43222222 36.18166667, 49.43222222 81.26805556, -56.15 81.26805556, -56.15 36.18166667))',
 'geospatial_vertical_max': '0',
 'geospatial_vertical_min': '-1850.0',
 'time_coverage_start': '1970-01-01T02:46:39',
 'time_coverage_end': '1970-01-01T02:46:39',
 'title': 'Radioactivity Monitoring of the Irish Marine Environment 1991 and 1992',
 'summary': '',
 'creator_name': '[{"creatorType": "author", "firstName": "A.", "lastName": "McGarry"}, {"creatorType": "author", "firstName": "S.", "lastName": "Lyons"}, {"creatorType": "author", "firstName": "C.", "lastName": "McEnri"}, {"creatorType": "author", "firstName": "T.", "lastName": "Ryan"}, {"creatorType": "author", "firstName": "M.", "lastName": "O\'Colmain"}, {"creatorType": "author", "firstName": "J.D.", "lastName": "Cunningham"}]',
 'keyw

In [ ]:
#| export
def get_species(dfs):
    

In [ ]:
#| export
def encode(fname_in, fname_out, nc_tpl_path, **kwargs):
    df = load_dump(fname_in)
    ref_ids = kwargs.get('ref_ids', df.ref_id.unique())
    print('Encoding ...')
    for ref_id in tqdm(ref_ids, leave=False):
        dfs = load_data(df, ref_id)
        print(get_fname(dfs))
        tfm = Transformer(dfs, cbs=[
            RemapRdnNameCB(),
            RenameColumnCB(),
            DropNAColumnsCB(),
            SanitizeDetectionLimitCB(),
            ReshapeLongToWide(),
            EncodeTimeCB(),
            SanitizeLonLatCB()
            ])
        
        # species_lut = get_maris_species(fname_in, 'species_helcom.pkl')
        # enums_xtra = {
        #     'species_t': {info['name']: info['id'] 
        #                   for info in species_lut.values() if info['name'] != ''}
        # }
        encoder = NetCDFEncoder(tfm(), 
                                src_fname=nc_tpl_path,
                                dest_fname=Path(fname_out) / get_fname(dfs), 
                                global_attrs=get_attrs(tfm, zotero_key=get_zotero_key(dfs), kw=kw),
                                verbose=kwargs.get('verbose', False)
                                # enums_xtra=enums_xtra
                                )
        encoder.encode()

In [ ]:
encode(fname_in, dir_dest, nc_tpl_path(), verbose=False, ref_ids=[ref_id])

Encoding ...


  0%|          | 0/1 [00:00<?, ?it/s]

191-OSPAR-ComissionÆs-Radioactive-Substances-Committee-(RSC)-2018.nc
Group: biota, Variable: sample
Group: biota, Variable: lon
Group: biota, Variable: lat
Group: biota, Variable: time
Group: biota, Variable: area
Group: biota, Variable: bio_group
Group: biota, Variable: species
Group: biota, Variable: body_part
Group: biota, Variable: c14_unc
Group: biota, Variable: c14_dl
Group: biota, Variable: c14_unit
Group: biota, Variable: k40_unc
Group: biota, Variable: k40_dl
Group: biota, Variable: k40_unit
Group: biota, Variable: tc99_unc
Group: biota, Variable: tc99_dl
Group: biota, Variable: tc99_unit
Group: biota, Variable: i131_unc
Group: biota, Variable: i131_dl
Group: biota, Variable: i131_unit
Group: biota, Variable: cs137_unc
Group: biota, Variable: cs137_dl
Group: biota, Variable: cs137_unit
Group: biota, Variable: pb210_unc
Group: biota, Variable: pb210_dl
Group: biota, Variable: pb210_unit
Group: biota, Variable: po210_unc
Group: biota, Variable: po210_dl
Group: biota, Variable: p

In [ ]:
encode(fname_in, dir_dest, nc_tpl_path(), verbose=False)

Encoding ...


  0%|          | 0/460 [00:00<?, ?it/s]

402-CCHDO-2018.nc


  0%|          | 1/460 [00:05<42:14,  5.52s/it]

374-Ístlund-et-al-1987.nc


...

 25%|██▍       | 114/460 [04:23<10:27,  1.81s/it]

158-Buesseler-2018.nc


 25%|██▌       | 115/460 [04:25<10:27,  1.82s/it]

404-Azouz-and-Dulai-2017.nc


KeyboardInterrupt: 